In [268]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [269]:
import datetime
import numpy as np
import pandas as pd
import os 
from pandas import ExcelFile
from keras import models 
from keras import layers
import tensorflow as tf
from tensorflow.keras import layers  #모듈(변수나 함수를 포함)만 불러오기
from sklearn.preprocessing import StandardScaler, MinMaxScaler  #표준 정규화, 최대-최소 정규화


import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc


In [270]:
pd.set_option('display.max_colwidth',-1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [271]:
pd.set_option('display.float_format', '{:.4f}'.format)    #과학적 표기법 안쓸래 
mpl.rcParams['axes.unicode_minus'] = False               # 마이너스 표기 오류 방지 

In [272]:
base_dir = "/content/gdrive/My Drive/Colab Notebooks/data/0928_final"
train_file = "2020train.csv"
test_file = "2020test.csv"


train_dir = os.path.join(base_dir,train_file)
test_dir = os.path.join(base_dir,test_file)

In [273]:
train_raw = pd.read_csv(train_dir, sep = "," , encoding = "cp949")   
test_raw = pd.read_csv(test_dir, sep = "," , encoding = "cp949")   

In [274]:
train1 = train_raw
test1 = test_raw

In [275]:
train1.info() # 89col , 32519 row

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35379 entries, 0 to 35378
Data columns (total 91 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    35379 non-null  int64  
 1   방송일시          35379 non-null  object 
 2   노출(분)         35379 non-null  float64
 3   마더코드          35379 non-null  int64  
 4   상품코드          35379 non-null  int64  
 5   상품명           35379 non-null  object 
 6   상품군           35379 non-null  object 
 7   판매단가          35379 non-null  int64  
 8   취급액           35379 non-null  int64  
 9   월             35379 non-null  int64  
 10  일             35379 non-null  int64  
 11  요일            35379 non-null  object 
 12  반기            35379 non-null  int64  
 13  분기            35379 non-null  object 
 14  365일          35379 non-null  int64  
 15  53주차          35379 non-null  int64  
 16  년월일           35379 non-null  object 
 17  시간            35379 non-null  object 
 18  시간hour        35379 non-nu

In [276]:
train_y = pd.DataFrame(train1['취급액boxcox'])

In [277]:
train_X = pd.DataFrame(train1, columns=['판매단가', '노출(분)',
'월', '일', '시간hour',
'168시간', '휴일', '방송시간', '매진여부',
'결제수단', '기온', '강수량', '풍속', '습도', '적설',
'전운량', '미세먼지', '초미세먼지', '판매단가순위',
'판매단가rank', '중분류순위', '소분류순위',
'방송내_순서', '방송순서', '판매상품종류수', '그룹코드_전체횟수', '그룹코드_대박횟수',
'그룹코드_대박확률','그룹코드_쪽박횟수', '그룹코드_쪽박확률',
'168시간_COS', '168시간_SIN', '24시간_COS', '24시간_SIN', '그룹코드_쪽박횟수', '그룹코드_쪽박확률',
'방송내_상품종류별_점수', '방송내_상품선호도'])

In [278]:
train_X =pd.get_dummies(train_X)

In [279]:
train_X.info()  # 40 col

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35379 entries, 0 to 35378
Data columns (total 40 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   판매단가          35379 non-null  int64  
 1   노출(분)         35379 non-null  float64
 2   월             35379 non-null  int64  
 3   일             35379 non-null  int64  
 4   시간hour        35379 non-null  int64  
 5   168시간         35379 non-null  int64  
 6   휴일            35379 non-null  int64  
 7   방송시간          35379 non-null  float64
 8   매진여부          35379 non-null  int64  
 9   기온            35379 non-null  float64
 10  강수량           35379 non-null  float64
 11  풍속            35379 non-null  float64
 12  습도            35379 non-null  float64
 13  적설            35379 non-null  float64
 14  전운량           35379 non-null  float64
 15  미세먼지          35379 non-null  float64
 16  초미세먼지         35379 non-null  float64
 17  판매단가순위        35379 non-null  int64  
 18  판매단가rank      35379 non-nu

In [280]:
train_data = train_X
train_targets = train_y

In [281]:
test_y = pd.DataFrame(test1['취급액boxcox'])

In [282]:
test_X = pd.DataFrame(test1, columns=['판매단가', '노출(분)',
'월', '일', '시간hour',
'168시간', '휴일', '방송시간', '매진여부',
'결제수단', '기온', '강수량', '풍속', '습도', '적설',
'전운량', '미세먼지', '초미세먼지', '판매단가순위',
'판매단가rank', '중분류순위', '소분류순위',
'방송내_순서', '방송순서', '판매상품종류수', '그룹코드_전체횟수', '그룹코드_대박횟수',
'그룹코드_대박확률','그룹코드_쪽박횟수', '그룹코드_쪽박확률',
'168시간_COS', '168시간_SIN', '24시간_COS', '24시간_SIN', '그룹코드_쪽박횟수', '그룹코드_쪽박확률',
'방송내_상품종류별_점수', '방송내_상품선호도'])

In [283]:
test_X =pd.get_dummies(test_X)

In [284]:
test_X.info()  # 40 col

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2716 entries, 0 to 2715
Data columns (total 40 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   판매단가          2716 non-null   int64  
 1   노출(분)         2716 non-null   float64
 2   월             2716 non-null   int64  
 3   일             2716 non-null   int64  
 4   시간hour        2716 non-null   int64  
 5   168시간         2716 non-null   int64  
 6   휴일            2716 non-null   int64  
 7   방송시간          2716 non-null   float64
 8   매진여부          2716 non-null   int64  
 9   기온            2716 non-null   float64
 10  강수량           2716 non-null   float64
 11  풍속            2716 non-null   float64
 12  습도            2716 non-null   float64
 13  적설            2716 non-null   float64
 14  전운량           2716 non-null   float64
 15  미세먼지          2716 non-null   float64
 16  초미세먼지         2716 non-null   float64
 17  판매단가순위        2716 non-null   int64  
 18  판매단가rank      2716 non-null 

In [285]:
test_data = test_X
test_targets = test_y

# scaling


In [286]:
# # min max scaler 

# mm_scaler = MinMaxScaler()

# train_data_scaled = mm_scaler.fit_transform(train_data)
# test_data_scaled = mm_scaler.transform(test_data)

# train_data = train_data_scaled
# test_data = test_data_scaled

In [287]:
## 정규화  : test도 train의 평균, 표준편차 값으로 정규화를 한다 ##
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [288]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution1D, Dropout
from keras.optimizers import SGD
from keras.initializers import random_uniform
from keras.callbacks import ModelCheckpoint, EarlyStopping

from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

import tensorflow.keras.backend as K
from tensorflow import nn
from tensorflow.keras.backend import shape
from tensorflow.keras.layers import Input, Dense, Flatten, Activation
from tensorflow.keras.models import Model, Sequential, load_model
from keras.layers import BatchNormalization
from tensorflow.keras.utils import get_custom_objects
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model


def Mish(x):
    return x * K.tanh(K.softplus(x))

get_custom_objects().update({'mish': Mish})

In [289]:
train_data_ar= np.asarray(train_data)
train_targets_ar = np.asarray(train_targets)

test_data_ar= np.asarray(test_data)
test_targets_ar = np.asarray(test_targets)

In [290]:
train_data_3d = train_data_ar.reshape((35379,40,1))
train_data_3d = np.nan_to_num(train_data_3d, copy=False) 

train_targets_3d = train_targets_ar.reshape((35379,1,1))
train_targets_3d = np.nan_to_num(train_targets_3d, copy=False) 


test_data_3d = test_data_ar.reshape((2716,40,1))
test_data_3d = np.nan_to_num(test_data_3d, copy=False) 

test_targets_3d = test_targets_ar.reshape((2716,1,1))
test_targets_3d = np.nan_to_num(test_targets_3d, copy=False) 

In [291]:

test_targets_3d = test_targets_ar.reshape((2716,1,1))
# define model

def build_model():
    model = Sequential()

    model.add(Convolution1D(filters=32, kernel_size=2, activation='relu', input_shape=(59,1) ))
    model.add(MaxPooling1D(pool_size=3))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))

    model.add(Flatten())
    model.add(Dense(32, activation='relu'))


    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mape', metrics=['mae']) 
    return model 
 

# 모델 평가 


In [ ]:

model = build_model()  # 케라스 모델 구성(컴파일 포함)

num_epochs = 1
BATCH_SIZE = 32
all_scores = []
all_mape_histories=[]

filename = 'checkpoint-epoch-{}-batch-{}-trial-001.h5'.format(num_epochs, BATCH_SIZE)
checkpoint = ModelCheckpoint(filename,             # file명을 지정합니다
                             monitor='val_loss',   # val_loss 값이 개선되었을때 호출됩니다
                             verbose=1,            # 로그를 출력합니다
                             save_best_only=True,  # 가장 best 값만 저장합니다
                             mode='auto'           # auto는 알아서 best를 찾습니다. min/max
                            )

earlystopping = EarlyStopping(monitor='val_loss',  # 모니터 기준 설정 (val loss) 
                              patience=10,         # N회 Epoch동안 개선되지 않는다면 종료
                             )

hist = model.fit(train_data_3d, train_targets_3d, 
      epochs=num_epochs, 
      batch_size=BATCH_SIZE)



# 테스트 MAPE


In [ ]:
 model.evaluate(test_data_3d, test_targets_3d)

In [ ]:
 y_pred = model.predict(test_data_3d, verbose=1)


In [ ]:
y_pred

# 취급액으로 변환한 MAPE


In [ ]:
def original_target(boxcox):
    return (0.2222222 * boxcox +1)**(1/0.2222222)

In [ ]:
y_pred_original =original_target(y_pred)
test_y_original =original_target(test_y)

In [ ]:
y_pred_df = pd.DataFrame(y_pred_original)
test_y_df = pd.DataFrame(test_y_original)


In [ ]:
# col 이름 맞춰주기 
y_pred_df.columns = ['취급액']
test_y_df.columns = ['취급액']

# 취급액으로 변환한 MAPE


In [ ]:
real_MAPE = ((abs(test_y_df - y_pred_df)/test_y_df).sum())/len(test_y_df)*100
print(real_MAPE)

#결과 엑셀 저장


In [ ]:
base_dir_excel = "/content/gdrive/My Drive/Colab Notebooks/result/CNN_result/y_pred"
base_dir_model = "/content/gdrive/My Drive/Colab Notebooks/result/model"

In [ ]:
now = datetime.datetime.now()

In [ ]:
excel_nm = "{}_epoch_{}_batch_{}.xlsx".format(now,num_epochs,BATCH_SIZE)

In [ ]:
xlxs_dir = os.path.join(base_dir_excel, excel_nm) 

In [ ]:
# keras.models.save_model(model, base_dir_model, overwrite=True, include_optimizer=True)
models.save_model(model, base_dir_model, overwrite=True, include_optimizer=True)

In [ ]:
y_pred_df.to_excel(xlxs_dir, # directory and file name to write

            sheet_name = 'Sheet1', 
            na_rep = 'NaN', 
            float_format = "%.2f", 
            header = True, 
            #columns = ["group", "value_1", "value_2"], # if header is False
            index = True, 
            index_label = "id", 
            startrow = 1, 
            startcol = 1, 
            #engine = 'xlsxwriter', 
            freeze_panes = (2, 0)
            ) 
